In [118]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.datasets import mnist
from torch.utils.data import DataLoader

vgg 的一个关键就是使用很多层 3 x 3 的卷积然后再使用一个最大池化层。

我们可以定义一个 vgg 的 block，传入三个参数，第一个是模型层数，第二个是输入的通道数，第三个是输出的通道数，第一层卷积接受的输入通道就是图片输入的通道数，然后输出最后的输出通道数，后面的卷积接受的通道数就是最后的输出通道数

In [119]:
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.ReLU(True)] # 定义第一层
    
    for i in range(num_convs-1): # 定义后面的很多层
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))
        
    net.append(nn.MaxPool2d(2, 2)) # 定义池化层
    return nn.Sequential(*net)

下面我们定义一个函数对这个 vgg block 进行堆叠

In [120]:
def vgg_stack(num_convs, channels):
    net = []
    for n, c in zip(num_convs, channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n, in_c, out_c))
    return nn.Sequential(*net)

我们定义一个稍微简单一点的 vgg 结构，其中有 3 个卷积层

In [121]:
vgg_net = vgg_stack((1, 2), ((1, 3), (3, 6)))
print(vgg_net)

In [122]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg, self).__init__()
        self.feature = vgg_net
        self.fc = nn.Sequential(
            nn.Linear(294, 16),
            nn.ReLU(True),
            nn.Linear(16, 10)
        )
    def forward(self, x):
#         print("shape1:"+str(x.shape))
        x = self.feature(x.float())
#         print("shape2:"+str(x.shape))
        x = x.view(x.shape[0], -1)
#         print("shape3:"+str(x.shape))
        x = self.fc(x)
#         print("shape4:"+str(x.shape))
        return x

In [123]:
from datetime import datetime

def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().data
    return num_correct / total


def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, label in train_data:
            if torch.cuda.is_available():
                im = Variable(im.cuda())  # (bs, 3, h, w)
                label = Variable(label.cuda())  # (bs, h, w)
            else:
                im = Variable(im)
                label = Variable(label)
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data
            train_acc += get_acc(output, label)

        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for im, label in valid_data:
                if torch.cuda.is_available():
                    im = Variable(im.cuda())
                    label = Variable(label.cuda())
                else:
                    im = Variable(im)
                    label = Variable(label)
                output = net(im)
                loss = criterion(output, label)
                valid_loss += loss.data
                valid_acc += get_acc(output, label)
            epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(valid_data),
                   valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)

In [124]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 数据预处理，标准化
    x=np.array([x.tolist()])
    x = torch.from_numpy(x)    
    return x

In [125]:
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据
train_set = mnist.MNIST('./data', train=True, transform=data_tf,download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

In [126]:
x=torch.tensor([])
for i,j in train_data:
#     print(i[0].shape) 
#     print(i[0])
    x=i
    break
print(x.shape)

In [127]:
x=Variable(x.cuda())
vgg_net=vgg_net.cuda()
x=vgg_net(x.float())
print(x.shape)
x = x.view(x.shape[0], -1)
print(x.shape)
fc = nn.Sequential(
        nn.Linear(294, 16),
        nn.ReLU(True),
        nn.Linear(16, 10)
        )
x = fc.cuda()(x)
print(x.shape)

In [128]:
net = vgg()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [129]:
train(net, train_data, test_data, 20, optimizer, criterion)